# StressGait Analysis - Saliva & Self-Reports

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font

from stressgait_analysis.dataset import StressGaitDataset

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True)

dataset

In [ ]:
dataset.cortisol_features

In [ ]:
fig, ax = plt.subplots()

bp.protocols.plotting.saliva_plot(
    data=dataset.cortisol,
    saliva_type="cortisol",
    sample_times=dataset.sample_times,
    test_times=[30, 38],
    test_title="GAIT",
    sample_times_absolute=True,
    ax=ax,
)

fig.tight_layout()

In [ ]:
dataset.cortisol_features.groupby(["condition", "saliva_feature"]).mean().unstack("condition")

In [ ]:
pg.pairwise_tests(
    data=dataset.cortisol_features.xs("max_inc", level="saliva_feature").reset_index(),
    dv="cortisol",
    between="condition",
)

In [ ]:
hue_order = ["control", "omc"]

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(data=dataset.cortisol.reset_index(), x="sample", y="cortisol", hue="condition", hue_order=hue_order, ax=ax)
sns.swarmplot(
    data=dataset.cortisol.reset_index(),
    x="sample",
    y="cortisol",
    hue="condition",
    hue_order=hue_order,
    ax=ax,
    palette=cmaps.faculties[:2],
    dodge=True,
)

fig.tight_layout()

In [ ]:
fig, axs = plt.subplots(figsize=(14, 5), ncols=2, sharey=True)

dataset.cortisol.unstack()["cortisol"].xs("control", level="condition").T.plot(ax=axs[0], legend=False, title="Control")
dataset.cortisol.unstack()["cortisol"].xs("omc", level="condition").T.plot(ax=axs[1], legend=False, title="OMC")

# sns.lineplot(data=dataset.cortisol.reset_index(), x="sample", y="cortisol", ax=ax)

fig.tight_layout()

In [ ]:
dataset.cortisol_features.unstack()

In [ ]:
fig, ax = bp.protocols.plotting.saliva_multi_feature_boxplot(
    data=dataset.cortisol_features,
    saliva_type="cortisol",
    features=["max_inc", "auc_g"],
    hue="condition",
    hue_order=["control", "omc"],
    legend_orientation="horizontal",
    legend_loc="upper center",
)

fig.tight_layout(rect=(0, 0, 1, 0.9))

In [ ]:
responder_list = dataset.cortisol.reindex(["S0", "S1"], level="sample").unstack().diff(axis=1).dropna(axis=1, how="all")
responder_list.columns = ["inc_S0S1"]
display(responder_list.groupby("condition").mean())

responder_list = responder_list > 1.5

display(responder_list)

responder_list.groupby("condition").value_counts()

In [ ]:
responder_list = dataset.cortisol_features.xs("max_inc", level="saliva_feature") > 1.5

display(responder_list)

responder_list.groupby("condition").value_counts()

In [ ]:
data = pd.read_csv(input_path.joinpath("stress_gait_labor.csv"))
data = data.assign(condition_fine=data["condition"].replace(condition_fine_mapping))
data = data.assign(condition=(data["condition_fine"].str.contains("OMC")).replace(condition_mapping))
data = data.set_index(["participant", "condition"])

data

In [ ]:
data = data.drop(index=["VP_03", "VP_04", "VP_09", "VP_14", "VP_21"], errors="ignore")

In [ ]:
data = convert_nan(data)
data.head()

In [ ]:
quest_dict = {
    "SSSQ-pre": data.filter(like="Pre_SSSQ").columns,
    "SSSQ-marker": data.filter(like="PreMocap_SSSQ").columns,
    "SSSQ-post": data.filter(like="Post_SSSQ").columns,
    "PANAS-pre": data.filter(like="Pre_PANAS").columns,
    "PANAS-marker": data.filter(like="PreMocap_PANAS").columns,
    "PANAS-post": data.filter(like="Post_PANAS").columns,
}

In [ ]:
quest_data = compute_scores(
    data=data,
    quest_dict=quest_dict,
    quest_kwargs={"PANAS-pre": {"language": "german"}, "PANAS-post": {"language": "german"}},
)

quest_data.head()

In [ ]:
panas_data_full = wide_to_long(quest_data, quest_name="PANAS", levels=["subscale", "time"])

omc_panas = panas_data_full.loc[pd.IndexSlice[:, "OMC", :, ["marker", "post"]]]
omc_panas = omc_panas.rename(index={"marker": "pre"})
control_panas = panas_data_full.loc[pd.IndexSlice[:, "Control", :, ["pre", "post"]]]

panas_data = pd.concat([omc_panas, control_panas]).sort_index()

panas_data.head()

In [ ]:
sssq_data_full = wide_to_long(quest_data, quest_name="SSSQ", levels=["subscale", "time"])

sssq_omc = sssq_data_full.loc[pd.IndexSlice[:, "OMC", :, ["marker", "post"]]]
sssq_omc = sssq_omc.rename(index={"marker": "pre"})
sssq_control = sssq_data_full.loc[pd.IndexSlice[:, "Control", :, ["pre", "post"]]]

sssq_data = pd.concat([sssq_omc, sssq_control]).sort_index()
sssq_data.head()

In [ ]:
for key, data in panas_data_full.xs("OMC", level="condition").groupby("subscale"):
    display(key)
    results = pg.pairwise_tests(data=data.reset_index(), dv="PANAS", subject="participant", within="time")
    display(results.round(3))

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(
    data=sssq_data_full.xs("OMC", level="condition").reset_index(),
    x="subscale",
    y="SSSQ",
    hue="time",
    hue_order=["pre", "marker", "post"],
    ax=ax,
)

fig.tight_layout()

In [ ]:
sssq_diff = (
    sssq_data.unstack()
    .reindex(["pre", "post"], level="time", axis=1)
    .diff(axis=1)
    .dropna(axis=1, how="all")
    .droplevel(level=-1, axis=1)
)

sssq_diff

In [ ]:
panas_diff = (
    panas_data.unstack()
    .reindex(["pre", "post"], level="time", axis=1)
    .diff(axis=1)
    .dropna(axis=1, how="all")
    .droplevel(level=-1, axis=1)
)

panas_diff

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(data=sssq_diff.reset_index(), y="SSSQ", x="subscale", hue="condition", hue_order=["Control", "OMC"], ax=ax)

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()

sns.boxplot(
    data=panas_diff.reset_index(), y="PANAS", x="subscale", hue="condition", hue_order=["Control", "OMC"], ax=ax
)

fig.tight_layout()

## Plot for ISB Abstract

In [ ]:
panas_data_full.to_csv(input_path.joinpath("panas_data.csv"))

In [ ]:
panas_data = pd.read_csv(input_path.joinpath("panas_data.csv"))
panas_data = panas_data.set_index(["participant", "condition", "subscale", "time"])
panas_data

In [ ]:
panas_data = panas_data.loc[pd.IndexSlice[:, "OMC", "NegativeAffect"]]

In [ ]:
from biopsykit.stats import StatsPipeline

In [ ]:
pipeline = StatsPipeline(
    steps=[("test", "pairwise_tests")], params={"dv": "PANAS", "subject": "participant", "within": "time"}
)

pipeline.apply(panas_data)

box_pairs, pvalues = pipeline.sig_brackets("test", stats_effect_type="within")

pipeline.display_results()

In [ ]:
fig, ax = plt.subplots()

bp.plotting.feature_boxplot(
    data=panas_data.reset_index(),
    x="time",
    y="PANAS",
    hue="time",
    show_legend=False,
    order=["pre", "marker", "post"],
    stats_kwargs={"pairs": box_pairs, "pvalues": pvalues, "verbose": 0},
    ax=ax,
)

fig.tight_layout()